# VacationPy
---


In [57]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from environs import Env
import numpy as np

# Load environment variables to get API key
env = Env()
env.read_env("../.env")
geoapify_key = env("GEOAPIFY_KEY", default=None)

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saipan,15.1355,145.7010,81.30,83,40,5.75,MP,1727716025
1,1,port-aux-francais,-49.3500,70.2167,35.33,90,100,34.23,TF,1727716025
2,2,edinburgh of the seven seas,-37.0676,-12.3116,56.53,85,58,21.45,SH,1727716025
3,3,walvis bay,-22.9575,14.5053,59.52,57,0,22.88,NaN,1727715860
4,4,blackmans bay,-43.0167,147.3167,55.72,65,93,1.99,AU,1727716025


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
coordinates_df = city_data_df[['Lat','Lng','Humidity']]

map_plot_1 = coordinates_df.hvplot.points(
    'Lng',
    'Lat', 
    geo=True, 
    tiles="OSM",
    size = "Humidity")

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [51]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 90
min_temp = 60

max_clouds = 75

evas_vacay_locations = city_data_df[(city_data_df['Max Temp'] < max_temp) & (city_data_df['Max Temp'] > min_temp)
                                    & (city_data_df['Cloudiness'] < max_clouds)]

# Drop any rows with null values
evas_vacay_locations = evas_vacay_locations.dropna()
# Display sample data
evas_vacay_locations



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saipan,15.1355,145.7010,81.30,83,40,5.75,MP,1727716025
7,7,port mathurin,-19.6833,63.4167,72.41,84,0,21.81,MU,1727716026
11,11,kasungu,-13.0333,33.4833,79.48,26,0,8.16,MW,1727716027
15,15,adamstown,-25.0660,-130.1015,65.82,67,33,6.62,PN,1727716027
24,24,yaren,-0.5472,166.9160,82.99,74,34,13.89,NR,1727716028
...,...,...,...,...,...,...,...,...,...,...
559,559,isangel,-19.5500,169.2667,73.72,88,37,2.01,VU,1727716067
560,560,papao,-17.7500,-149.5500,78.48,73,28,10.69,PF,1727716113
565,565,mount isa,-20.7333,139.5000,74.97,31,0,16.11,AU,1727716114
569,569,imbituva,-25.2300,-50.6044,89.78,19,0,4.36,BR,1727716115


### Step 3: Create a new DataFrame called `hotel_df`.

In [54]:
# Create a new  DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = evas_vacay_locations.loc[:,['City','Country',  'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame for storing the hlotel information
hotel_df['Hotel Name'] = np.nan

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,saipan,MP,15.1355,145.7010,83,NaN
7,port mathurin,MU,-19.6833,63.4167,84,NaN
11,kasungu,MW,-13.0333,33.4833,26,NaN
15,adamstown,PN,-25.0660,-130.1015,67,NaN
24,yaren,NR,-0.5472,166.9160,74,NaN
...,...,...,...,...,...,...
559,isangel,VU,-19.5500,169.2667,88,NaN
560,papao,PF,-17.7500,-149.5500,73,NaN
565,mount isa,AU,-20.7333,139.5000,31,NaN
569,imbituva,BR,-25.2300,-50.6044,19,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [60]:
# Set parameters to search for a hotel
radius =50000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    response = response.json()
  

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saipan - nearest hotel: Chalan Kanoa Beach Hotel
port mathurin - nearest hotel: Escale Vacances
kasungu - nearest hotel: Kasungu Inn
adamstown - nearest hotel: No hotel found
yaren - nearest hotel: Aiwo Hotel
la'ie - nearest hotel: No hotel found
hannibal - nearest hotel: Best Western On the River
sao joao da barra - nearest hotel: Pousada Mediterrâneo
cockburn town - nearest hotel: The Salt Raker Inn
gebeit - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
gamba - nearest hotel: No hotel found
acarau - nearest hotel: No hotel found
inarajan village - nearest hotel: Leo Palace
pointe aux piments - nearest hotel: Hotel Villas Mon Plaisir
ardakan - nearest hotel: هتل رستوران باغ سرای اردکان
micheweni - nearest hotel: Bomani Guesthouse
pacasmayo - nearest hotel: Playa Linda
georgetown - nearest hotel: Page 63 hostel
newman - nearest hotel: Hampton Inn & Suites
santiago de cao - nearest hotel: Hostal El Encanto
waingapu - nearest hotel: HOTEL

,City,Country,Lat,Lng,Humidity,Hotel Name
0,saipan,MP,15.1355,145.7010,83,Chalan Kanoa Beach Hotel
7,port mathurin,MU,-19.6833,63.4167,84,Escale Vacances
11,kasungu,MW,-13.0333,33.4833,26,Kasungu Inn
15,adamstown,PN,-25.0660,-130.1015,67,No hotel found
24,yaren,NR,-0.5472,166.9160,74,Aiwo Hotel
...,...,...,...,...,...,...
559,isangel,VU,-19.5500,169.2667,88,Tanna Lodge
560,papao,PF,-17.7500,-149.5500,73,Hiti Moana Villa Lodge
565,mount isa,AU,-20.7333,139.5000,31,Ibis Styles
569,imbituva,BR,-25.2300,-50.6044,19,Pousada Fazenda Virá


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [63]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles='OSM', 
    hover_cols=['Hotel Name', 'Country']
    )

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)